In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Yolov5/yolov5

/content/drive/MyDrive/Colab Notebooks/Yolov5/yolov5


In [7]:
%pip install git+https://github.com/Cupcakus/pafy

  Cloning https://github.com/Cupcakus/pafy to /tmp/pip-req-build-c1ez_r4g
  Running command git clone -q https://github.com/Cupcakus/pafy /tmp/pip-req-build-c1ez_r4g
  Created wheel for pafy: filename=pafy-0.5.5-py2.py3-none-any.whl size=35757 sha256=3f1d7fde1aebd51b0dde531981d0896e6ed6a1364d6e1aea04fb687ce2d81b2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1nz1hyce/wheels/93/1a/52/3c75e2f35b4aac122bcb0a64c08339224a76eafef3f150b9db
Successfully built pafy


In [8]:
%pip install youtube-dl==2020.12.2

In [ ]:
import torch
import numpy as np
import cv2
import pafy
from time import time


class DroneDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using OpenCV.
    """
    
    def __init__(self, url, out_file, model_name):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = url
        self.model = self.load_model(model_name)
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("\n\nDevice Used:",self.device)


    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        play = pafy.new(self._URL).streams[-1]
        assert play is not None
        return cv2.VideoCapture(play.url)


    def load_model(self, model_name):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        if model_name:
            #model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_name, force_reload=True)
            model = torch.hub.load('/content/drive/MyDrive/Colab Notebooks/Yolov5/yolov5', 'custom', path=model_name, source='local')
        else:
            model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model


    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
     
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        return labels, cord


    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]


    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame


    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        while True:
            start_time = time()
            ret, frame = player.read()
            if not ret:
                break
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)



# Create a new object and execute.
detection = DroneDetection("https://www.youtube.com/watch?v=sOdOO4uim5s", "video_detect.avi",  model_name='runs/train/exp/weights/best1.pt')
detection()

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /content/drive/MyDrive/Colab Notebooks/Yolov5/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-2-16 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 




Device Used: cuda
Frames Per Second : 3.9682539682539684
Frames Per Second : 25.641025641025642
Frames Per Second : 27.77777777777778
Frames Per Second : 27.027027027027028
Frames Per Second : 27.77777777777778
Frames Per Second : 25.641025641025642
Frames Per Second : 27.77777777777778
Frames Per Second : 27.77777777777778
Frames Per Second : 26.315789473684212
Frames Per Second : 27.77777777777778
Frames Per Second : 24.390243902439025
Frames Per Second : 27.027027027027028
Frames Per Second : 27.77777777777778
Frames Per Second : 27.77777777777778
Frames Per Second : 26.315789473684212
Frames Per Second : 25.0
Frames Per Second : 26.315789473684212
Frames Per Second : 27.77777777777778
Frames Per Second : 27.027027027027028
Frames Per Second : 26.315789473684212
Frames Per Second : 25.641025641025642
Frames Per Second : 27.77777777777778
Frames Per Second : 25.0
Frames Per Second : 27.027027027027028
Frames Per Second : 27.027027027027028
Frames Per Second : 27.027027027027028
Fra